In [54]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences


In [76]:
with open('data/shakespeare_proc_rnn.txt', 'r') as f:
    source = f.read()

num_chars = len(set(source))
char_seqs = [source[i-40:i+1] for i in range(40, len(source))]
char_to_int = {}
for i, char in enumerate(set(source)):
    char_to_int[char] = i
encoded = np.array([[char_to_int[char] for char in seq] for seq in char_seqs])

In [ ]:
X, y = encoded[:,:-1], encoded[:,-1]
X = np.array([to_categorical(x, num_classes=num_chars) for x in X])
y = to_categorical(y, num_classes=num_chars)
    
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(num_chars, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy')
model.fit(X, y, epochs=50)
model.save('rnn_model')

In [72]:
model = load_model('rnn_model')

text = 'shall i compare thee to a summer\'s day?\n'
for i in range(580):
    encoded = [char_to_int[char] for char in text]
    encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
    encoded = to_categorical(encoded, num_classes=len(char_to_int))
    pred = model.predict_classes(encoded, verbose=0)
    out_char = ''
    for char, index in char_to_int.items():
        if index == pred:
            out_char = char
            break
    text += char

print(text)

shall i compare thee to a summer's day?
the the sear the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love the love 
